## Homework 1
#### DS7337: Natural Language Processing, Fall 2022
By: Allen Hoskins

In [45]:
import nltk
from nltk.corpus import gutenberg
from nltk.tokenize import word_tokenize
from nltk import text
import numpy as np
from sklearn.preprocessing import MinMaxScaler,minmax_scale
from IPython.display import Image
from IPython.core.display import HTML 
import requests
import urllib.request
import time
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from string import punctuation
import pandas as pd

In [26]:
#### Method to get the Vocab Size and normalize the score

def n_vocab_size(*arg):
    vocab_size = np.array([])
    vocab_size_norm = np.array([])
    
    #### Getting the Vocab Size
    for text in arg:
        vocab_size = np.append(vocab_size,len(set(text)))
    
    #### Normalizing using the formula 
    for vsize in vocab_size:
        vocab_size_norm = np.append(vocab_size_norm,(vsize - vocab_size.min()) /
                                                    (vocab_size.max() - vocab_size.min()))
    
    #### Normalizing using sklearn preprocessing 
    vocab_size_norm_sklearn = minmax_scale(vocab_size, feature_range=(0,1), axis=0)
    
    return(vocab_size,vocab_size_norm,vocab_size_norm_sklearn)

In [15]:
from nltk.corpus import gutenberg

In [19]:
from nltk.book import *

*** Introductory Examples for the NLTK Book ***
Loading text1, ..., text9 and sent1, ..., sent9
Type the name of the text or sentence to view it.
Type: 'texts()' or 'sents()' to list the materials.
text1: Moby Dick by Herman Melville 1851
text2: Sense and Sensibility by Jane Austen 1811
text3: The Book of Genesis
text4: Inaugural Address Corpus
text5: Chat Corpus
text6: Monty Python and the Holy Grail
text7: Wall Street Journal
text8: Personals Corpus
text9: The Man Who Was Thursday by G . K . Chesterton 1908


In [27]:
vocab_size = n_vocab_size(text1,text2,text3,text4,text5,text6,text7,text8,text9)

In [28]:
print("Normalized Values by using the formula", *vocab_size[1],sep='\n- ')

Normalized Values by using the formula
- 1.0
- 0.31440496457795597
- 0.09231698610577187
- 0.48970289417321106
- 0.2722829370091713
- 0.05810313581196112
- 0.6205722444944808
- 0.0
- 0.31297709923664124


In [29]:
print("Normalized Values by using sklearn", *vocab_size[2], sep='\n- ')

Normalized Values by using sklearn
- 0.9999999999999999
- 0.3144049645779559
- 0.09231698610577188
- 0.489702894173211
- 0.2722829370091713
- 0.05810313581196112
- 0.6205722444944807
- 0.0
- 0.3129770992366412


# Section 2: 
>After consulting section 3.2 in chapter 1 of Bird-Klein, create a method for scoring the long-word vocabulary size of a text, and likewise normalize (and explain) the scoring as in step 1 above.

In [11]:
HW2_TEXT = gutenberg.words('austen-emma.txt')
HW1_URL = 'https://www.gutenberg.org/cache/epub/24644/pg24644.txt'

In [32]:
def url_text(url):
    req = requests.get(url).text
    resp = nltk.word_tokenize(req)
    return nltk.Text(resp)
    
def long_words(text,is_url=False):
    if is_url == True:
        unique_text = set(url_text(text))
        long_text = [w for w in unique_text if len(w)> 15]
        return sorted(long_text)
    else:
        unique_text = set(text)
        long_text = [w for w in unique_text if len(w)> 15]
        return sorted(long_text)
def text_freq_dist(url):
    unique_text = set(url_text(url))
    fdist = FreqDist(url_text(url))
    return sorted(w for w in unique_text if len(w) > 8 and fdist[w] > 7)

In [33]:
long_words(HW1_URL,is_url=True)

['//www.gutenberg.org/2/4/6/4/24644/',
 'Daffy-down-dilly',
 'trademark/copyright',
 'unenforceability',
 'www.gutenberg.org',
 'www.gutenberg.org/contact',
 'www.gutenberg.org/donate',
 'www.gutenberg.org/license']

In [35]:
long_words(HW2_TEXT,is_url=False)

['Disingenuousness',
 'companionableness',
 'disagreeableness',
 'disinterestedness',
 'incomprehensible',
 'misunderstanding',
 'misunderstandings',
 'unceremoniousness',
 'undistinguishing',
 'unseasonableness']

In [36]:
text_freq_dist(url=HW1_URL)

['Foundation',
 'Gutenberg',
 'Gutenberg-tm',
 'Illustration',
 'agreement',
 'associated',
 'copyright',
 'donations',
 'electronic',
 'paragraph',
 'trademark']

# Section 3:
> Now create a “text difficulty score” by combining the lexical diversity score from homework 1, and your normalized score of vocabulary size and long-word vocabulary size, in equal weighting. Explain what you see when this score is applied to same graded texts you used in homework 1.

In [53]:
def lexical_diversity(text):
    return len(set(text)) / len(text)

In [86]:
s1 = pd.Series(['HW1_URL',lexical_diversity(HW1_URL),'fix',len(long_words(HW1_URL,is_url=True)),len(HW1_URL)])
s2 = pd.Series(['HW2_Text',lexical_diversity(HW2_TEXT),'fix',len(long_words(HW2_TEXT,is_url=False)),len(HW2_TEXT)])

In [87]:
df = pd.DataFrame([list(s1), list(s2)],columns= ["Text", "Lexical Diversity",'Normalized Score','Long Words',"Length"])

In [88]:
df

,Text,Lexical Diversity,Normalized Score,Long Words,Length
0,HW1_URL,0.388889,fix,8,54
1,HW2_Text,0.040592,fix,10,192427


In [67]:
df_vocab_size = n_vocab_size(HW1_URL,HW2_TEXT)

In [77]:
print("Normalized Values by using the formula", *df_vocab_size[0],sep='\n- ')

Normalized Values by using the formula
- 21.0
- 7811.0


In [79]:
df_vocab_size[0]

array([  21., 7811.])